In [ ]:
pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
import faiss

# Data Collection

In [ ]:
import pickle

data_path = '/content/drive/MyDrive/homeworks/ФТИАД/CB'
with open(data_path+'/texts_cb.pkl','rb') as f:
    source_data = pickle.load(f)

source_data[:15]

['Мы тут планируем обновить Альфа-Мобайл для Windows Phone и устроили в FB небольшой опрос: ',
 'Выиграйте поездку на Олимпийские Игры 2016 в Рио с Visa + 150 000, 100 000 или 50 000 миль с картой Visa Alfa-Miles: ',
 'Если пользуетесь ВКонтакте, обратите внимание на этот пост: ',
 '10 лет - это не шутки! У нашего интернет-банка «Альфа-Клик» юбилей. Поздравления принимаем ;) ',
 'Первыми из российских банков сделали витрину продуктов и услуг прямо во vkontakte ',
 'Знаете ли вы, что можно вернуть налог за покупку квартиры, лечение, обучение, ипотеку? Поучаствуйте в нашем опросе: ',
 'Встречайте первое из трех запланированных обновлений Альфа-Мобайла для Windows Phone: ',
 'Неизвестная часть саги о финансах Гарри :) ',
 'Чем хорош онлайн-чат в Альфа-Мобайле? Там в любое время суток ответят профессиональные и очень веселые ребята ;-) ',
 'Друзья! Обратите внимание на график работы офисов банка в праздники: ',
 'Дорогие клиенты, сердечно поздравляем вас с 71-й годовщиной Победы в Великой 

In [ ]:
data_path = '/content/drive/MyDrive/homeworks/ФТИАД/CB'
with open(data_path+'/texts_encoded.pkl','rb') as f:
    embedings = pickle.load(f)

embedings[:15]

array([[ 6.69153035e-01,  6.21956810e-02, -9.39231515e-01, ...,
        -4.21812981e-01,  2.13465527e-01, -6.45891428e-02],
       [ 4.82775599e-01,  1.15565889e-01, -8.60237479e-01, ...,
        -5.16844749e-01,  5.34010708e-01,  3.64922616e-03],
       [ 5.60642004e-01, -5.90740405e-02, -9.51622069e-01, ...,
        -1.19195633e-01, -9.97867465e-01, -5.47595031e-04],
       ...,
       [ 1.60805687e-01,  1.73223153e-01, -7.42895365e-01, ...,
        -4.60402459e-01, -9.96495068e-01,  6.64633811e-02],
       [ 3.60883594e-01,  6.70840032e-03, -8.84056985e-01, ...,
        -4.81867164e-01, -9.97484624e-01,  3.39360908e-02],
       [-9.17937830e-02,  1.11725159e-01, -7.62385428e-01, ...,
        -4.02019233e-01, -9.93297815e-01, -1.93332471e-02]])

# Faiss

In [ ]:
import faiss
import numpy as np

class FaissKMeans:
    def __init__(self, n_clusters=4, n_init=10, max_iter=500):
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.max_iter = max_iter
        self.kmeans = None
        self.cluster_centers_ = None
        self.inertia_ = None

    def fit(self, X):
        self.kmeans = faiss.Kmeans(d=X.shape[1],
                                   k=self.n_clusters,
                                   niter=self.max_iter,
                                   nredo=self.n_init)
        self.kmeans.train(X.astype(np.float32))
        self.cluster_centers_ = self.kmeans.centroids
        self.inertia_ = self.kmeans.obj[-1]

        return self.kmeans.centroids

    def predict(self, X):
        return self.kmeans.index.search(X.astype(np.float32), 1)[1]

In [ ]:
from sklearn.metrics import silhouette_score
from tqdm import tqdm

dct = {}

for i in tqdm(np.arange(2,16)):

    glist =FaissKMeans(n_clusters = int(i))
    glist.fit(X=embedings)

    clusters = glist.predict(embedings)

    cluster = unzip(clusters)

    emb_marked['y'] = cluster
    emb_marked.columns = emb_marked.columns.astype(str)

    dct[i] = silhouette_score(emb_marked[emb_marked.columns[:-1]], emb_marked['y'])

dct

100%|██████████| 14/14 [43:56<00:00, 188.30s/it]


{2: 0.496148450973612,
 3: 0.20726273643737061,
 4: 0.207933475971777,
 5: 0.16683933804893977,
 6: 0.16203304830698187,
 7: 0.1456196062282202,
 8: 0.14266356117501056,
 9: 0.1419904673732355,
 10: 0.1331387997279784,
 11: 0.12580818711271613,
 12: 0.1279360713136685,
 13: 0.123251291077456,
 14: 0.12191791658158177,
 15: 0.11910269129297656}

In [ ]:
def unzip(lst):
    res = []
    for i in lst:
        res.append(i[0])
    return res

In [ ]:
model = FaissKMeans()
centroids = model.fit(embeddings)
clusters = model.predict(embeddings)

In [ ]:
data_path = '/content/drive/MyDrive/homeworks/ФТИАД/CB'
with open(data_path+"centroids.npy", 'wb') as f:
  np.save(f, centroids)

In [ ]:
cluster = unzip(clusters)

In [ ]:
emb_marked = pd.DataFrame(embedings)
emb_marked['y'] = cluster
emb_marked.columns = emb_marked.columns.astype(str)
emb_marked.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,y
0,0.669153,0.062196,-0.939232,-0.275068,-0.499306,-0.237098,-0.560503,-0.257521,-0.102171,-0.172746,...,0.219931,-0.166507,-0.030150,-0.617421,-0.033062,0.767442,-0.421813,0.213466,-0.064589,2
1,0.482776,0.115566,-0.860237,-0.042523,-0.636484,-0.142353,-0.512147,-0.086268,-0.356728,-0.216151,...,0.088638,-0.123630,-0.213836,0.574210,0.006328,0.620624,-0.516845,0.534011,0.003649,3
2,0.560642,-0.059074,-0.951622,-0.097045,-0.216248,-0.278038,-0.386376,-0.030050,-0.282619,-0.067468,...,0.057935,-0.056402,-0.065967,-0.706944,-0.295879,0.395146,-0.119196,-0.997867,-0.000548,2
3,0.727838,-0.058848,-0.852882,-0.232654,-0.103353,-0.315110,-0.269721,-0.198935,-0.222619,0.013290,...,0.029249,0.050721,0.217139,-0.550059,0.062202,0.784680,-0.149174,-0.935130,0.016512,2
4,0.683901,0.050998,-0.939855,-0.368929,-0.365766,-0.143491,-0.285856,-0.028148,-0.278661,-0.094100,...,-0.001602,-0.120186,-0.060469,-0.157612,-0.050325,0.290321,-0.189618,-0.972422,-0.041965,2


In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(emb_marked[emb_marked.columns[:-1]], emb_marked['y'])

0.207933475971777

# Demention Reduction

In [ ]:
%%time

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(emb_marked[:20000])
tsne.kl_divergence_

CPU times: user 4min 24s, sys: 757 ms, total: 4min 25s
Wall time: 4min 26s


2.9353787899017334

In [ ]:
%%time
import plotly.express as px
fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color=cluster[:20000])
fig.update_layout(
    title="t-SNE visualization of Economical News Clustering",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

CPU times: user 1.04 s, sys: 119 ms, total: 1.15 s
Wall time: 1.82 s


In [ ]:
emb_marked.to_pickle(data_path+"/embeddings_clusters.pkl")